In [6]:
import os
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC

# 저장 폴더 및 CSV 파일 경로 설정
folder_name = "인사행정"
os.makedirs(folder_name, exist_ok=True)
csv_file_path = os.path.join(folder_name, "files.csv")

# CSV 파일 생성 및 헤더 작성
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["fileName", "path"])

# Chrome 옵션 설정 (다운로드 경로 설정)
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": os.path.abspath(folder_name),  # 다운로드 경로 설정
    "download.prompt_for_download": False,  # 다운로드 시 사용자 확인 제거
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  # PDF 파일을 브라우저 내에서 열지 않고 직접 다운로드
})
options.page_load_strategy = 'eager'  # 또는 'normal'을 사용

# 웹드라이버 시작
driver = webdriver.Chrome(options=options)

try:
    # 1페이지부터 작업을 시작합니다.
    driver.get("https://www.korea.kr/briefing/pressReleaseList.do")

    wait = WebDriverWait(driver, 10)

    period_select_element = wait.until(EC.presence_of_element_located((By.ID, "period")))
    period_select = Select(period_select_element)
    period_select.select_by_value("direct")  # "직접입력" 옵션 선택

    start_date_input = wait.until(EC.presence_of_element_located((By.ID, "srchStartDate")))
    driver.execute_script("arguments[0].value = '2019-01-01';", start_date_input)

    end_date_input = wait.until(EC.presence_of_element_located((By.ID, "srchEndDate")))
    driver.execute_script("arguments[0].value = '2024-08-26';", end_date_input)

    search_box = wait.until(EC.presence_of_element_located((By.ID, "srchKeyword")))
    driver.execute_script("arguments[0].value = '인사행정';", search_box)

    driver.execute_script("srchMain();")

    last_successful_link = None

    while True:
        results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.list_type > ul > li > a")))

        for i in range(len(results)):
            try:
                results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.list_type > ul > li > a")))
                first_link_url = results[i].get_attribute('href')

                if not first_link_url.startswith("http"):
                    first_link_url = "https://www.korea.kr" + first_link_url

                print(f"({i+1}/{len(results)}) 검색 결과 링크: {first_link_url}")
                driver.get(first_link_url)

                file_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "filedown")))
                pdf_links = file_section.find_elements(By.CSS_SELECTOR, "a > img[alt='PDF파일']")

                if pdf_links:
                    for pdf_link in pdf_links:
                        pdf_a_tag = pdf_link.find_element(By.XPATH, "./ancestor::a")
                        pdf_file_name = pdf_a_tag.text.strip()
                        pdf_url = pdf_a_tag.get_attribute('href')

                        pdf_a_tag.click()
                        time.sleep(5)

                        relative_path = os.path.join(folder_name, pdf_file_name)

                        with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
                            writer = csv.writer(file)
                            writer.writerow([pdf_file_name, relative_path])

                        print(f"다운로드할 PDF 파일 이름: {pdf_file_name}, 저장 위치: {relative_path}")

                    last_successful_link = first_link_url

                else:
                    print("PDF 파일을 찾을 수 없습니다.")

                driver.back()
                time.sleep(2)

            except Exception as e:
                print(f"오류가 발생했습니다: {e}")
                if last_successful_link:
                    print(f"마지막 성공한 링크로 다시 시도 중: {last_successful_link}")
                    driver.get(last_successful_link)
                    time.sleep(3)
                    continue

        try:
            # 다음 페이지 번호를 찾아 이동 (단순히 1페이지씩 이동)
            current_page = driver.find_element(By.CSS_SELECTOR, "span.num.on")
            next_page = current_page.find_element(By.XPATH, "following-sibling::span[@class='num']/a")
            driver.execute_script("arguments[0].click();", next_page)
            time.sleep(3)

        except Exception:
            try:
                # 다음 10 페이지로 넘어가는 'next' 버튼을 찾고 클릭
                next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(3)
            except Exception as e:
                print(f"다음 페이지로 이동할 수 없습니다. 작업이 완료되었거나 더 이상 페이지가 없습니다. 이유: {e}")
                break

    print("브라우저가 곧 종료됩니다. 종료를 막으려면 Enter 키를 누르세요.")
    input()

except Exception as e:
    print(f"오류가 발생했습니다: {e}")
finally:
    driver.quit()


(1/20) 검색 결과 링크: https://www.korea.kr/briefing/pressReleaseView.do?newsId=156647156&pageIndex=1&repCodeType=&repCode=&startDate=2019-01-01&endDate=2024-08-26&srchWord=%EC%9D%B8%EC%82%AC%ED%96%89%EC%A0%95&period=direct
다운로드할 PDF 파일 이름: 213 [특허청] 케이(K)-특허행정 필리핀에 전파한다!.pdf, 저장 위치: 인사행정/213 [특허청] 케이(K)-특허행정 필리핀에 전파한다!.pdf
(2/20) 검색 결과 링크: https://www.korea.kr/briefing/pressReleaseView.do?newsId=156647133&pageIndex=1&repCodeType=&repCode=&startDate=2019-01-01&endDate=2024-08-26&srchWord=%EC%9D%B8%EC%82%AC%ED%96%89%EC%A0%95&period=direct
다운로드할 PDF 파일 이름: (보도자료) 법무부 양성평등정책담당관 임용 (배포즉시보도).pdf, 저장 위치: 인사행정/(보도자료) 법무부 양성평등정책담당관 임용 (배포즉시보도).pdf
(3/20) 검색 결과 링크: https://www.korea.kr/briefing/pressReleaseView.do?newsId=156647098&pageIndex=1&repCodeType=&repCode=&startDate=2019-01-01&endDate=2024-08-26&srchWord=%EC%9D%B8%EC%82%AC%ED%96%89%EC%A0%95&period=direct
다운로드할 PDF 파일 이름: 0826 찾아가는 ‘경복궁 별빛야행’, 「궁, 바퀴를 달다」(붙임2).pdf, 저장 위치: 인사행정/0826 찾아가는 ‘경복궁 별빛야행’, 「궁, 바퀴를 달다」(붙임2).pdf
(4/20) 검색 결과 링크: https:/